<a href="https://colab.research.google.com/github/linyuehzzz/5526_neural_networks/blob/master/parity_m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Lab 1**
This code implements a two-layer perceptron with the backpropagation algorithm to solve the parity problem. Using a momentum term for learning rate control.  
Yue Lin (lin.3326 at osu.edu)  
Created: 9/26/2020

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#### **Set up libraries**

In [16]:
from random import seed, randrange, uniform, shuffle
from math import exp
import csv
import copy

#### **Generate training data**

In [17]:
def generate_data(n_inputs, n):
  data = []
  for i in range(n):
    x = []
    for j in range(n_inputs):
      x.append(randrange(2))
    if x.count(1) % 2 == 0:
      x.append(0)
    else:
      x.append(1)
    data.append(x)
  return data

#### **Initialize network**

In [18]:
def init_net(n_inputs, n_hidden, n_outputs):
  net = list()
  hidden_layer = [{'w':[uniform(-1, 1) for i in range(n_inputs + 1)], 
                   'c':[0 for i in range(n_inputs + 1)]} 
                  for i in range(n_hidden)]
  net.append(hidden_layer)
  output_layer = [{'w':[uniform(-1, 1) for i in range(n_hidden + 1)],
                   'c':[0 for i in range(n_inputs + 1)]} 
                  for i in range(n_outputs)]
  net.append(output_layer)
  return net

#### **Define activation function**

In [19]:
def transfer(w, inputs):
  # activation function: logistic sigmoid with a = 1
  v = w[-1]
  for i in range(len(w)-1):
    v += w[i] * inputs[i]
  phi = 1. / (1. + exp(-v))
  return phi

#### **Forward propagate**

In [20]:
def fwd_prop(net, row):
  inputs = row
  for layer in net:
    new_inputs = []
    for neuron in layer:
      neuron['y'] = transfer(neuron['w'], inputs)
      new_inputs.append(neuron['y'])
    inputs = new_inputs
  return inputs

#### **Backward propagate error**

In [21]:
def bwd_prop_err(net, d):
  for i in reversed(range(len(net))):
    layer = net[i]
    errors = list()
    if i != len(net)-1:
      for j in range(len(layer)):
        error = 0.
        for neuron in net[i + 1]:
          error += (neuron['w'][j] * neuron['delta'])
        errors.append(error)
    else:
      for j in range(len(layer)):
        neuron = layer[j]
        errors.append(d[j] - neuron['y'])
    
    for j in range(len(layer)):
      neuron = layer[j]
      phi_d = neuron['y'] * (1. - neuron['y'])
      neuron['delta'] = errors[j] * phi_d

#### **Update weights**

In [36]:
def update_weights(net, row, lr, alpha):
  for i in range(len(net)):
    inputs = row[:-1]
    if i != 0:
      inputs = [neuron['y'] for neuron in net[i - 1]]
    for neuron in net[i]:
      for j in range(len(inputs)):
        neuron['w'][j] += lr * neuron['delta'] * inputs[j] + alpha * neuron['c'][j]
        neuron['c'][j] = lr * neuron['delta'] * inputs[j] + alpha * neuron['c'][j]
      neuron['w'][-1] += lr * neuron['delta'] + alpha * neuron['c'][-1]
      neuron['c'][-1] = lr * neuron['delta'] + alpha * neuron['c'][-1]

#### **Train**

In [23]:
def train(net, data, lr, alpha, n_outputs):
  epoch = 0
  c = True
  
  # Train
  while c:
    epoch += 1
    sum_error = 0
    max_abs_error = 0
    c = False
    
    # Randomization
    shuffle(data)

    # Online learning
    for row in data:
      outputs = fwd_prop(net, row)
      d = row[-n_outputs:]
      bwd_prop_err(net, d)
      update_weights(net, row, lr, alpha)
      
      # Stopping criteria
      sum_error += sum([(d[i]-outputs[i])**2 for i in range(n_outputs)])
      if sum([abs(d[i]-outputs[i]) for i in range(n_outputs)]) > max_abs_error:
        max_abs_error = sum([abs(d[i]-outputs[i]) for i in range(n_outputs)])
      if max_abs_error > 0.05:
        c = True
    print('>epoch=%d, lrate=%.3f, sum_error=%.3f, max_abs_error=%.3f' 
          % (epoch, lr, sum_error, max_abs_error))

#### **Wrapper**

Initialize

In [24]:
%cd "/content/gdrive/My Drive/Colab Notebooks/cse5526"
n_inputs = 4  # Number of input neurons
n_hidden = 4  # Number of hidden neurons
n_outputs = 1 # Number of output neurons

# Read training data
with open("lab1-train.csv", "r") as f:
  data = []
  for line in f:
      data_line = line.rstrip().split(",")
      data_line = [int(i) for i in data_line] 
      data.append(data_line)
print(data)

# Initialize network
net = init_net(n_inputs, n_hidden, n_outputs)
for layer in net:
	print(layer)

/content/gdrive/My Drive/Colab Notebooks/cse5526
[[0, 0, 0, 0, 0], [1, 0, 0, 0, 1], [0, 1, 0, 0, 1], [0, 0, 1, 0, 1], [0, 0, 0, 1, 1], [1, 1, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 1, 0], [1, 1, 1, 0, 1], [1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [1, 1, 1, 1, 0]]
[{'w': [0.31025455389476186, -0.530280346740019, -0.13145262777309408, 0.6894259929863558, 0.7120990928406843], 'c': [0, 0, 0, 0, 0]}, {'w': [-0.4256177518266626, -0.9006391130495479, 0.3806213156939955, 0.3996276492239996, 0.6795059335429632], 'c': [0, 0, 0, 0, 0]}, {'w': [0.7704485061774766, -0.09080120365921274, 0.24558817574709635, 0.5816017624856809, -0.48640460697412347], 'c': [0, 0, 0, 0, 0]}, {'w': [0.7403329117832156, 0.861950289801368, -0.14882222772496267, 0.894085415305663, -0.19213968054772512], 'c': [0, 0, 0, 0, 0]}]
[{'w': [0.06959451135705486, -0.08134472946497784, 0.15824049183046096, -0.2680006196420921, 0.8793750536927958], 'c': [0, 0, 0, 0, 0]}]


Print initialized weights

In [37]:
net = [ [{'w': [0.023751897144906442, 0.5439181010827554, 0.7949939709979394, 
                0.5575722128060299, -0.0833224505320811], 'c': [0, 0, 0, 0, 0]}, 
         {'w': [-0.42994177241818776, 0.21188832212031072, 0.8223560150844227, 
                0.8774539980092704, 0.31816289627719896], 'c': [0, 0, 0, 0, 0]}, 
         {'w': [0.1714638576679366, 0.6385710100729274, -0.715278099957847, 
                0.08266885375715938, -0.22296262982508863], 'c': [0, 0, 0, 0, 0]}, 
         {'w': [-0.8943826130758457, 0.07003766731452088, -0.9945608691130916, 
                0.028556706695230805, -0.17212852084981978], 'c': [0, 0, 0, 0, 0]}],
         [{'w': [-0.6504348668157289, 0.1059724636195516, -0.4211953496188985, 
                 -0.7266794062610493, -0.8039881319466737], 'c': [0, 0, 0, 0, 0]}]]
for layer in net:
	print(layer)

[{'w': [0.023751897144906442, 0.5439181010827554, 0.7949939709979394, 0.5575722128060299, -0.0833224505320811], 'c': [0, 0, 0, 0, 0]}, {'w': [-0.42994177241818776, 0.21188832212031072, 0.8223560150844227, 0.8774539980092704, 0.31816289627719896], 'c': [0, 0, 0, 0, 0]}, {'w': [0.1714638576679366, 0.6385710100729274, -0.715278099957847, 0.08266885375715938, -0.22296262982508863], 'c': [0, 0, 0, 0, 0]}, {'w': [-0.8943826130758457, 0.07003766731452088, -0.9945608691130916, 0.028556706695230805, -0.17212852084981978], 'c': [0, 0, 0, 0, 0]}]
[{'w': [-0.6504348668157289, 0.1059724636195516, -0.4211953496188985, -0.7266794062610493, -0.8039881319466737], 'c': [0, 0, 0, 0, 0]}]


Train

In [ ]:
lr = 0.05
alpha = 0.9

# Train
inet = copy.deepcopy(net)
train(inet, data, lr, alpha, n_outputs)
for layer in inet:
	print(layer)